# Notebook Testing Harness

This notebook executes all Jupyter notebooks in the project recursively and reports their completion status.

It serves as a testing harness to verify that all notebooks run successfully without errors.

In [1]:
from aips.spark import get_spark_session

import os
import glob
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from datetime import datetime
import sys
import traceback
from pathlib import Path
import json
from tqdm.notebook import tqdm

spark = get_spark_session()

from aips import set_engine; set_engine("opensearch")

## Configuration

In [2]:
KERNEL_NAME = "python3"
KERNEL_OVERRIDES = {"1.open-information-extraction.ipynb": "ch5-spacy"}
EXCLUDE_DIRS = [".ipynb_checkpoints", "__pycache__"]
EXPORT_RESULTS = False
EXCLUDED_NOTEBOOKS = ["bonus.related-terms-from-documents.ipynb"
                      "bonus.phrase-detection.ipynb",
                      "bonus.phrase-detection.ipynb",
                      "bonus.related-terms-from-documents.ipynb",
                      "a.defunct.synthesize-search-sessions.ipynb",
                      "a.synthesize-search-sessions.ipynb",
                      "a.generate-movie-embeddings.ipynb",
                      "welcome.ipynb",
                      "aips-test-suite.ipynb",
                      "4.train-upload-search-ltr.ipynb",
                      "ch13-tokenizer-analysis.ipynb"]

## Utility Functions

In [3]:
def format_time(seconds):
    if seconds < 60:
        formatted = f"{seconds:.2f}s"
    elif seconds < 3600:
        formatted =  f"{seconds / 60:.2f}m"
    else:
        formatted =  f"{seconds / 3600:.2f}h"
    return formatted

def export_results(results, filename="test_results.json"):
    if results:
        with open(filename, 'w') as f:
            json.dump({"timestamp": datetime.now().isoformat(),
                       "results": results["details"],
                       "summary": results["summary"]}, f, indent=2)
        print(f"📄 Results exported to {filename}")

In [4]:
def get_notebook_files(root_dir=".", exclude_dirs=None, excluded_notebooks=None):
    """
    Recursively get all .ipynb files in the specified directory and its subdirectories,
    excluding specific directories and notebooks.
    
    Args:
        root_dir (str): Root directory to search from
        exclude_dirs (list): List of directory names to exclude
        exclude_notebooks (list): List of notebook filenames to exclude
        
    Returns:
        list: Sorted list of notebook file paths
    """
    if exclude_dirs is None:
        exclude_dirs = EXCLUDE_DIRS
    
    if excluded_notebooks is None:
        excluded_notebooks = EXCLUDED_NOTEBOOKS
        
    notebook_files = []
    
    for directory in os.walk(root_dir):
        for path in Path(directory[0]).rglob("*.ipynb"):
            if any(exclude_dir in str(path) for exclude_dir in exclude_dirs) or \
                path.name in excluded_notebooks:
                continue
            if str(path) not in notebook_files:
                notebook_files.append(str(path))
    
    notebook_files = sorted(notebook_files)
    return list(map(Path, notebook_files))

In [5]:
def execute_notebook(notebook_path, timeout=600, kernel_name="python3"):
    start_time = datetime.now()
    
    try:
        with open(notebook_path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
            ep = ExecutePreprocessor(timeout=timeout, kernel_name=kernel_name)
            ep.preprocess(nb, {'metadata': {'path': os.path.dirname(notebook_path)}})
        execution_time = (datetime.now() - start_time).total_seconds()
        return True, execution_time, None
    
    except Exception as e:
        execution_time = (datetime.now() - start_time).total_seconds()
        error_type = type(e).__name__
        error_msg = str(e)
        # Get traceback for detailed error information
        tb = traceback.format_exc()
        
        return False, execution_time, {
            'type': error_type,
            'message': error_msg,
            'traceback': tb
        }

## Main Testing Function

In [ ]:
def run_test_harness(exclude_dirs=None, exclude_notebooks=None, stop_on_failure=True,
                     chapter_to_run=None, verbose_errors=True):
    root_dir = "."
    timeout = 600 
    print(f"🔍 Notebook Testing Harness - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📁 Testing from root directory: {os.path.abspath(root_dir)}")
    print("=" * 80)

    notebook_files = get_notebook_files(root_dir, exclude_dirs, exclude_notebooks)

    if chapter_to_run:
        notebook_files = [f for f in notebook_files if (chapter_to_run in str(f))]

    print(f"📋 Found {len(notebook_files)} notebook(s) to test.")
    for nb_file in notebook_files:
        rel_path = os.path.relpath(str(nb_file), root_dir)
        print(f"   • {rel_path}")
    print()
    
    results = {
        'details': [],
        'summary': {
            'total': len(notebook_files),
            'successful': 0,
            'failed': 0,
            'total_time': 0
        }
    }
    
    print("🚀 Executing notebooks:")
    for notebook_path in tqdm(notebook_files, desc="Progress", colour="purple"):
        rel_path = os.path.relpath(str(notebook_path), root_dir)
        print(f"\n📔 Testing: {rel_path}")
        if "checkpoint" in str(notebook_path):
            print(f"\n📔 Skipping: {rel_path}")
            continue
        kernel_name = KERNEL_OVERRIDES.get(notebook_path.name, "python3")
        success, execution_time, error = execute_notebook(str(notebook_path), timeout, kernel_name)
        
        results['summary']['total_time'] += execution_time
        
        if success:
            print(f"   ✅ SUCCESS - Completed in {format_time(execution_time)}")
            results['summary']['successful'] += 1
        else:
            print(f"   ❌ FAILED - Error after {format_time(execution_time)}")
            print(f"      Error: {error['type']}")
            results['summary']['failed'] += 1
        
        results['details'].append({
            'notebook': rel_path,
            'success': success,
            'execution_time': execution_time,
            'error': error
        })
        if not success and stop_on_failure:
            print("Terminating test run due to test failure.")
            break
    
    print("\n" + "=" * 80)
    print(f"📊 SUMMARY:")
    print(f"   Total notebooks tested: {results['summary']['total']}")
    print(f"   ✅ Successful: {results['summary']['successful']}")
    if results['summary']['failed'] > 0:
        print(f"   ❌ Failed: {results['summary']['failed']}")
    
    success_rate = (results['summary']['successful'] / results['summary']['total']) * 100
    print(f"   Success rate: {success_rate:.1f}%")
    print(f"   Total execution time: {format_time(results['summary']['total_time'])}")

    if results['summary']['failed'] > 0:
        print(f"\n❌ FAILED NOTEBOOKS:")
        for result in results['details']:
            if not result['success']:
                if verbose_errors:
                    print(f"   • {result['notebook']}: {result['error']['type']}: {result['error']['message']}")
                else:
                    print(f"   • {result['notebook']}: {result['error']['type']}")
    
    print(f"\n🏁 Testing completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    if EXPORT_RESULTS and results:
        export_results(results, "AIPS_test_resulsts.json")
    
    return results

In [ ]:
# Last cell in chapter 3 will fail for most non-solr notebookes without {!funct} translation capabilities
results = run_test_harness(chapter_to_run="ch03")

🔍 Notebook Testing Harness - 2025-09-07 23:16:36
📁 Testing from root directory: /home/jovyan
📋 Found 4 notebook(s) to test.
   • chapters/ch14/1.question-answering-visualizer.ipynb
   • chapters/ch14/2.question-answering-data-preparation.ipynb
   • chapters/ch14/3.question-answering-fine-tuning.ipynb
   • chapters/ch14/4.question-answering-demo-application.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/4 [00:00<?, ?it/s]


📔 Testing: chapters/ch14/1.question-answering-visualizer.ipynb
   ✅ SUCCESS - Completed in 8.63s

📔 Testing: chapters/ch14/2.question-answering-data-preparation.ipynb
   ✅ SUCCESS - Completed in 4.68m

📔 Testing: chapters/ch14/3.question-answering-fine-tuning.ipynb
   ✅ SUCCESS - Completed in 36.39m

📔 Testing: chapters/ch14/4.question-answering-demo-application.ipynb
   ✅ SUCCESS - Completed in 19.44s

📊 SUMMARY:
   Total notebooks tested: 4
   ✅ Successful: 4
   Success rate: 100.0%
   Total execution time: 41.54m

🏁 Testing completed at 2025-09-07 23:58:08


In [8]:
results = run_test_harness(chapter_to_run="ch04")

🔍 Notebook Testing Harness - 2025-09-07 23:58:08
📁 Testing from root directory: /home/jovyan
📋 Found 2 notebook(s) to test.
   • chapters/ch04/1.setting-up-the-retrotech-dataset.ipynb
   • chapters/ch04/2.signals-boosting.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/2 [00:00<?, ?it/s]


📔 Testing: chapters/ch04/1.setting-up-the-retrotech-dataset.ipynb
   ✅ SUCCESS - Completed in 22.69s

📔 Testing: chapters/ch04/2.signals-boosting.ipynb
   ✅ SUCCESS - Completed in 49.99s

📊 SUMMARY:
   Total notebooks tested: 2
   ✅ Successful: 2
   Success rate: 100.0%
   Total execution time: 1.21m

🏁 Testing completed at 2025-09-07 23:59:21


In [9]:
results = run_test_harness(chapter_to_run="ch05")

🔍 Notebook Testing Harness - 2025-09-07 23:59:21
📁 Testing from root directory: /home/jovyan
📋 Found 3 notebook(s) to test.
   • chapters/ch05/1.open-information-extraction.ipynb
   • chapters/ch05/2.index-datasets.ipynb
   • chapters/ch05/3.semantic-knowledge-graph.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/3 [00:00<?, ?it/s]


📔 Testing: chapters/ch05/1.open-information-extraction.ipynb
   ✅ SUCCESS - Completed in 9.10s

📔 Testing: chapters/ch05/2.index-datasets.ipynb
   ✅ SUCCESS - Completed in 55.42s

📔 Testing: chapters/ch05/3.semantic-knowledge-graph.ipynb
   ✅ SUCCESS - Completed in 1.04m

📊 SUMMARY:
   Total notebooks tested: 3
   ✅ Successful: 3
   Success rate: 100.0%
   Total execution time: 2.11m

🏁 Testing completed at 2025-09-08 00:01:28


In [10]:
results = run_test_harness(chapter_to_run="ch06")

🔍 Notebook Testing Harness - 2025-09-08 00:01:28
📁 Testing from root directory: /home/jovyan
📋 Found 3 notebook(s) to test.
   • chapters/ch06/1.skg-classification-disambiguation.ipynb
   • chapters/ch06/2.related-keywords-from-signals.ipynb
   • chapters/ch06/3.spell-correction.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/3 [00:00<?, ?it/s]


📔 Testing: chapters/ch06/1.skg-classification-disambiguation.ipynb
   ✅ SUCCESS - Completed in 32.83s

📔 Testing: chapters/ch06/2.related-keywords-from-signals.ipynb
   ✅ SUCCESS - Completed in 3.38m

📔 Testing: chapters/ch06/3.spell-correction.ipynb
   ✅ SUCCESS - Completed in 3.27m

📊 SUMMARY:
   Total notebooks tested: 3
   ✅ Successful: 3
   Success rate: 100.0%
   Total execution time: 7.20m

🏁 Testing completed at 2025-09-08 00:08:40


In [11]:
results = run_test_harness(chapter_to_run="ch07")

🔍 Notebook Testing Harness - 2025-09-08 00:08:40
📁 Testing from root directory: /home/jovyan
📋 Found 2 notebook(s) to test.
   • chapters/ch07/1.index-datasets.ipynb
   • chapters/ch07/2.semantic-search.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/2 [00:00<?, ?it/s]


📔 Testing: chapters/ch07/1.index-datasets.ipynb
   ✅ SUCCESS - Completed in 1.31m

📔 Testing: chapters/ch07/2.semantic-search.ipynb
   ✅ SUCCESS - Completed in 25.76s

📊 SUMMARY:
   Total notebooks tested: 2
   ✅ Successful: 2
   Success rate: 100.0%
   Total execution time: 1.74m

🏁 Testing completed at 2025-09-08 00:10:24


In [12]:
results = run_test_harness(chapter_to_run="ch08")

🔍 Notebook Testing Harness - 2025-09-08 00:10:24
📁 Testing from root directory: /home/jovyan
📋 Found 1 notebook(s) to test.
   • chapters/ch08/1.signals-boosting.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/1 [00:00<?, ?it/s]


📔 Testing: chapters/ch08/1.signals-boosting.ipynb
   ❌ FAILED - Error after 4.41m
      Error: CellExecutionError
Terminating test run due to test failure.

📊 SUMMARY:
   Total notebooks tested: 1
   ✅ Successful: 0
   ❌ Failed: 1
   Success rate: 0.0%
   Total execution time: 4.41m

❌ FAILED NOTEBOOKS:
   • chapters/ch08/1.signals-boosting.ipynb: CellExecutionError: An error occurred while executing the following cell:
------------------
from aips.data_loaders import index_time_boosts

boosts_collection = engine.get_collection("normalized_signals_boosts")
boosted_products_collection = \
    engine.get_collection("products_with_signals_boosts")

boosted_products = index_time_boosts.load_dataframe(
    products_collection,
    boosts_collection)

boosted_products_collection.write(boosted_products)
------------------

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
Cell In[23], line 1

In [ ]:
results = run_test_harness(chapter_to_run="ch09")

🔍 Notebook Testing Harness - 2025-09-08 00:14:49
📁 Testing from root directory: /home/jovyan
📋 Found 2 notebook(s) to test.
   • chapters/ch09/1.personalization.ipynb
   • chapters/ch09/2.embedding-based-personalization.ipynb

🚀 Executing notebooks:


Progress:   0%|          | 0/2 [00:00<?, ?it/s]


📔 Testing: chapters/ch09/1.personalization.ipynb


KeyboardInterrupt: 

In [ ]:
results = run_test_harness(chapter_to_run="ch10")

In [ ]:
results = run_test_harness(chapter_to_run="ch11")

In [ ]:
results = run_test_harness(chapter_to_run="ch12")

In [ ]:
results = run_test_harness(chapter_to_run="ch13")

In [ ]:
results = run_test_harness(chapter_to_run="ch14")

In [ ]:
results = run_test_harness(chapter_to_run="ch15")

## Export Results (Optional)